In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from plotly.graph_objs import *
init_notebook_mode(connected=True)
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col
from linearmodels import IV2SLS
import numpy as np
from linearmodels.iv import IV2SLS as ivregress
plt.rcParams["figure.figsize"] = (20, 25)
plt.rcParams["xtick.labelsize"] = 16
plt.rcParams["ytick.labelsize"] = 16
plt.rcParams["axes.labelsize"] = 20
plt.rcParams['legend.fontsize'] = 20
%matplotlib inline

# Question 1

Let's get some conversion out of the way

_We know that 1500g is 3 Pounds 5 Ounces._

As such, 1 ounce is **approx** 28.34952g - for numerical purposes, lets just round this to 28.35g

I constructed 10 bins as there was a range of about 10 ounces in our data.

From the graph below, we can see that there is an **inverse relationship** between mortality and birth weight.

That is to say, as birth weight increases, the mortality rate dereases.

It is interesting, however, to see a kink occur at the cutoff point!

In [2]:
birth_df = pd.read_stata("birthweight.dta")

In [3]:
# encoding treatment status
birth_df['treatment'] = birth_df['bweight'].apply(lambda x: 1 if x < 1500 else 0)
birth_df[["bweight","treatment"]].tail()

,bweight,treatment
376403,1503,0
376404,1389,1
376405,1418,1
376406,1418,1
376407,1446,1


In [4]:
# creation of 1 ounce bins.
bins = [1350, 1386, 1414, 1443, 1471, 1500, 1528, 1556, 1585, 1613, 1651]
labels = ['<1386', '<1414', '<1443', '<1471', '<1500', '<1528', '<1556', '<1585', '<1613', '<1651']
birth_df["birth_bins_1_oz"] = pd.cut(birth_df['bweight'], bins=bins, labels=labels, right=False)

In [5]:
# Head of the Data
birth_df[["bweight",'birth_bins_1_oz']].head()

,bweight,birth_bins_1_oz
0,1361,<1386
1,1361,<1386
2,1361,<1386
3,1361,<1386
4,1361,<1386


In [6]:
# Tail of the Data
birth_df[["bweight",'birth_bins_1_oz']].tail()

,bweight,birth_bins_1_oz
376403,1503,<1528
376404,1389,<1414
376405,1418,<1443
376406,1418,<1443
376407,1446,<1471


In [7]:
# Plot Mortality vs Birthweight
trace_0_data = birth_df[birth_df['bweight'] >= 1500].groupby("birth_bins_1_oz").mean()["agedth5"].values
ounces_0 = list(birth_df[birth_df['bweight'] >= 1500].groupby("birth_bins_1_oz").mean()["agedth5"].index)

trace_1_data = birth_df[birth_df['bweight'] < 1500].groupby("birth_bins_1_oz").mean()["agedth5"].values
ounces_1 = list(birth_df[birth_df['bweight'] < 1500].groupby("birth_bins_1_oz").mean()["agedth5"].index)

trace0 = go.Scatter(
    x = ounces_0,
    y = trace_0_data,
    name = 'Mortality Rate',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 4)
)

trace1 = go.Scatter(
x = ["<1500","<1500"],
y = [0.045, 0.077],
name = "Cut-Off = 1500g",
    line = dict(
        color = ('rgb(22, 96, 167)'),
        width = 4,))

trace2 = go.Scatter(
    x = ounces_1,
    y = trace_1_data,
    name = 'Mortality Rate',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 4)
)

traces = [trace0, trace1, trace2]

layout = dict(title = f'One-Year Mortality Rate By Birth Weight - 1 Oz Bin',
              xaxis = dict(title = 'Weight (grams)'),
              yaxis = dict(title = f'Average Mortality Rate'))

fig = dict(data=traces, layout=layout)

iplot(fig, filename='styled-line')

# Question 2

Windening the window to 3 ounces results in us only have 4 windows.

This smoothes out our plot as we are hiding the variability of our data in bigger intervals.

We still see an **inverse relationship** - however, this plot is significantly more linear!

In [8]:
# creation of 3 ounce bins.
bins = [1350, 1416, 1500, 1585, 1651]
labels = ['<1416', '<1500',  '<1585', '<1651']
birth_df["birth_bins_3_oz"] = pd.cut(birth_df['bweight'], bins=bins, labels=labels, right=False)

In [9]:
# Plot Mortality vs Birthweight
trace_0_data = birth_df[birth_df['bweight'] >= 1500].groupby("birth_bins_3_oz").mean()["agedth5"].values
ounces_0 = list(birth_df[birth_df['bweight'] >= 1500].groupby("birth_bins_3_oz").mean()["agedth5"].index)

trace_1_data = birth_df[birth_df['bweight'] < 1500].groupby("birth_bins_3_oz").mean()["agedth5"].values
ounces_1 = list(birth_df[birth_df['bweight'] < 1500].groupby("birth_bins_3_oz").mean()["agedth5"].index)

trace0 = go.Scatter(
    x = ounces_0,
    y = trace_0_data,
    name = 'Mortality Rate',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 4)
)

trace1 = go.Scatter(
x = ["<1500","<1500"],
y = [0.045, 0.077],
name = "Cut-Off = 1500g",
    line = dict(
        color = ('rgb(22, 96, 167)'),
        width = 4,))

trace2 = go.Scatter(
    x = ounces_1,
    y = trace_1_data,
    name = 'Mortality Rate',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 4)
)

traces = [trace0, trace1, trace2]

layout = dict(title = f'One-Year Mortality Rate By Birth Weight - 3 Oz Bin',
              xaxis = dict(title = 'Weight (grams)'),
              yaxis = dict(title = f'Average Mortality Rate'))

fig = dict(data=traces, layout=layout)

iplot(fig, filename='styled-line')

# Question 3

By looking at the data around the cutoff point, it seems that mothers in both the treatment and control groups exhibit almost identical behaviours. In this manner, there is no evidence to demonstrate the birth weight is maniputable by the mothers.

However, more broadly and perhaps outside the scope of this assignment, behaviours like smoking or drinking during a pregnancy can cause major changes in birthweight - in this manner, birthweight would a vector for manipulation.

# Question 4

I selected gestation and prenatal visits as pre-treatment covariates. The plots below show that these two covariates violate the **continuity restriction**.

Additional covariates should have little impact on estimates if we have speciﬁed the functional form correctly. Furthermore, the addition of covariates can help improve precision by reducing the standard errors.

In this instance, both of these covariates would need to be included in our regressions in order to control for them.


In [10]:
# Plot Mothers Education vs Birthweight
trace_0_data = birth_df[birth_df['bweight'] >= 1500].groupby("birth_bins_1_oz").mean()["gest"].values
ounces_0 = list(birth_df[birth_df['bweight'] >= 1500].groupby("birth_bins_1_oz").mean()["gest"].index)

trace_1_data = birth_df[birth_df['bweight'] < 1500].groupby("birth_bins_1_oz").mean()["gest"].values
ounces_1 = list(birth_df[birth_df['bweight'] < 1500].groupby("birth_bins_1_oz").mean()["gest"].index)

trace0 = go.Scatter(
    x = ounces_0,
    y = trace_0_data,
    name = 'Gestation (Weeks)',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 4)
)

trace1 = go.Scatter(
x = ["<1500","<1500"],
y = [30, 33],
name = "Cut-Off = 1500g",
    line = dict(
        color = ('rgb(22, 96, 167)'),
        width = 4,))

trace2 = go.Scatter(
    x = ounces_1,
    y = trace_1_data,
    name = 'Gestation (Weeks)',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 4)
)

traces = [trace0, trace1, trace2]

layout = dict(title = f'Gestation By Birth Weight - 1 Oz Bin',
              xaxis = dict(title = 'Weight (grams)'),
              yaxis = dict(title = f'Average Gestation (Weeks)'))

fig = dict(data=traces, layout=layout)

iplot(fig, filename='styled-line')

In [11]:
# Plot Mothers Education vs Birthweight
trace_0_data = birth_df[birth_df['bweight'] >= 1500].groupby("birth_bins_1_oz").mean()["nprenatal"].values
ounces_0 = list(birth_df[birth_df['bweight'] >= 1500].groupby("birth_bins_1_oz").mean()["nprenatal"].index)

trace_1_data = birth_df[birth_df['bweight'] < 1500].groupby("birth_bins_1_oz").mean()["nprenatal"].values
ounces_1 = list(birth_df[birth_df['bweight'] < 1500].groupby("birth_bins_1_oz").mean()["nprenatal"].index)

trace0 = go.Scatter(
    x = ounces_0,
    y = trace_0_data,
    name = 'Average Prenatal Visits',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 4)
)

trace1 = go.Scatter(
x = ["<1500","<1500"],
y = [8, 10],
name = "Cut-Off = 1500g",
    line = dict(
        color = ('rgb(22, 96, 167)'),
        width = 4,))

trace2 = go.Scatter(
    x = ounces_1,
    y = trace_1_data,
    name = 'Average Prenatal Visits',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 4)
)

traces = [trace0, trace1, trace2]

layout = dict(title = f'Prenatal Visits By Birth Weight - 1 Oz Bin',
              xaxis = dict(title = 'Weight (grams)'),
              yaxis = dict(title = f'Average Visits'))

fig = dict(data=traces, layout=layout)

iplot(fig, filename='styled-line')

# Question 5

The regression is of the following form:

$Y_i = \alpha_0 + \alpha_1 Treatment + \alpha_2 BirthWeight + \epsilon$

- $\alpha_1$ represents the effect of the treatment


- $\alpha_2$ represents the slope for **both** of the lines in the RDD.

From the regression out below we see that $\alpha_1 = -0.0071$ and $\alpha_2 = -0.0001$

In [12]:
birth_df['const'] = 1

FEATURES = ['const', "treatment", "bweight"]

model_1 = sm.OLS(birth_df['agedth5'], birth_df[FEATURES], missing="drop")
results_1 = model_1.fit()
print(results_1.summary2())

                  Results: Ordinary least squares
Model:              OLS              Adj. R-squared:     0.002      
Dependent Variable: agedth5          AIC:                -30565.6421
Date:               2018-03-07 14:10 BIC:                -30533.1268
No. Observations:   376408           Log-Likelihood:     15286.     
Df Model:           2                F-statistic:        290.2      
Df Residuals:       376405           Prob (F-statistic): 1.13e-126  
R-squared:          0.002            Scale:              0.053983   
----------------------------------------------------------------------
             Coef.    Std.Err.      t       P>|t|     [0.025    0.975]
----------------------------------------------------------------------
const        0.2637     0.0133    19.8477   0.0000    0.2377    0.2898
treatment   -0.0071     0.0015    -4.6814   0.0000   -0.0101   -0.0041
bweight     -0.0001     0.0000   -15.9791   0.0000   -0.0002   -0.0001
-----------------------------------------

# Question 6

The regression is of the following form:

$Y_i = \alpha_0 + \alpha_1 Treatment + \alpha_2 Treatment(BirthWeight-1500) + \alpha_3(1-Treatment)(BirthWeight - 1500) + \epsilon$

- $\alpha_1$ represents the effect of the treatment


- $\alpha_2$ represents the slope of the regression line for those in the treatment group.


- $\alpha_3$ represents the slope of the regression line for those in the control group

**Note:** 

- `treatment` $=\alpha_1$


- `treat_bweight_1500` $=\alpha_2$


- `no_treat_bweight_1500` $=\alpha_3$

In [13]:
birth_df['bweight_1500'] = birth_df['bweight'] - 1500
birth_df['treat_bweight_1500'] = birth_df['bweight_1500'] * birth_df['treatment']
birth_df['no_treatment'] = 1 - birth_df['treatment']
birth_df['no_treat_bweight_1500'] = birth_df['bweight_1500'] * birth_df['no_treatment']

FEATURES = ["const","treatment","treat_bweight_1500", "no_treat_bweight_1500"]

caliper_50 = birth_df[(birth_df['bweight'] >= 1450) & (birth_df['bweight'] <= 1550)]

model_2 = sm.OLS(caliper_50['agedth5'], caliper_50[FEATURES], missing="drop")
results_2 = model_2.fit()
print(results_2.summary2())

                   Results: Ordinary least squares
Model:                OLS              Adj. R-squared:     0.000     
Dependent Variable:   agedth5          AIC:                -8121.4335
Date:                 2018-03-07 14:10 BIC:                -8082.9455
No. Observations:     111525           Log-Likelihood:     4064.7    
Df Model:             3                F-statistic:        16.55     
Df Residuals:         111521           Prob (F-statistic): 9.53e-11  
R-squared:            0.000            Scale:              0.054436  
---------------------------------------------------------------------
                       Coef.  Std.Err.    t    P>|t|   [0.025  0.975]
---------------------------------------------------------------------
const                  0.0651   0.0014 45.5202 0.0000  0.0623  0.0679
treatment             -0.0168   0.0036 -4.6053 0.0000 -0.0240 -0.0097
treat_bweight_1500    -0.0003   0.0001 -2.9432 0.0032 -0.0006 -0.0001
no_treat_bweight_1500 -0.0004   0.0001 

# Question 7

Please see the comparison table below.

The wider the caliper, the larger the treatment effect (less negative) becomes.

This is because the wider our caliper, the more opportunity for confoudning variables to influence our estimate.

In general, we would like our caliper to be as small as possible so as to ensure that we are really looking at datapoints close to the cut off point. After all, this is the entire intuition behind the RDD design.

While the 30g caliper provides the smallest, window of observation, I would say that this should be the caliper of choice.

However, the 100g caliper has a lower standard error - although, not much smaller than 30g.

In [14]:
caliper_30 = birth_df[(birth_df['bweight'] >= 1470) & (birth_df['bweight'] <= 1530)]

model_3 = sm.OLS(caliper_30['agedth5'], caliper_30[FEATURES], missing="drop")
results_3 = model_3.fit()

caliper_100 = birth_df[(birth_df['bweight'] >= 1400) & (birth_df['bweight'] <= 1600)]

model_4 = sm.OLS(caliper_100['agedth5'], caliper_100[FEATURES], missing="drop")
results_4 = model_4.fit()

results_table = summary_col(results=[results_3,results_2,results_4],
                            float_format='%0.4f',
                            stars = True,
                            model_names=['30g Caliper',
                                         '50g Caliper',
                                         '100g Caliper'])
results_table.add_title('OLS Regressions')

print(results_table)

                     OLS Regressions
                      30g Caliper 50g Caliper 100g Caliper
----------------------------------------------------------
const                 0.0674***   0.0651***   0.0604***   
                      (0.0016)    (0.0014)    (0.0011)    
treatment             -0.0214***  -0.0168***  -0.0076***  
                      (0.0052)    (0.0036)    (0.0020)    
treat_bweight_1500    -0.0005**   -0.0003***  -0.0002***  
                      (0.0002)    (0.0001)    (0.0000)    
no_treat_bweight_1500 -0.0009***  -0.0004***  -0.0001***  
                      (0.0001)    (0.0001)    (0.0000)    
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01


# Part II

# Question 1

We have the following:

- 798 Never Takers ->  $Z=1, D=0$


- 581 compliers -> $Z = 1, D=1$


- 1097 never-takers/compliers -> $Z = 0, D=0$


- No Defiers observed


- No Always Takers Observed

In [15]:
mto_df = pd.read_stata("MTO_DT_subset.dta")
pd.crosstab(index=mto_df["assignment"],  # Make a crosstab
                              columns=mto_df["treated"])

treated,0.0,1.0
assignment,,
0.0,1097,0
1.0,798,581


# Question 2

I would not interpret my estimate as a causal effect as there seems to be some selection on non-observables.

Through treatment alone, we do not see a statistically significant impact on outcome.

In [16]:
def encode(x):
    if x[0] == "0":
        return 0
    else:
        return 1

In [17]:
mto_df['diabetes'] = mto_df['diabetes'].apply(lambda x: encode(x)).astype("float64")
mto_df['const'] = 1

FEATURES = ['const', "treated"]

model_1 = sm.OLS(mto_df['diabetes'], mto_df[FEATURES], missing="drop")
results_1 = model_1.fit()
print(results_1.summary2())

                 Results: Ordinary least squares
Model:              OLS              Adj. R-squared:     -0.000   
Dependent Variable: diabetes         AIC:                3402.4611
Date:               2018-03-07 14:10 BIC:                3414.0899
No. Observations:   2476             Log-Likelihood:     -1699.2  
Df Model:           1                F-statistic:        0.2960   
Df Residuals:       2474             Prob (F-statistic): 0.586    
R-squared:          0.000            Scale:              0.23119  
--------------------------------------------------------------------
             Coef.    Std.Err.      t      P>|t|     [0.025   0.975]
--------------------------------------------------------------------
const        0.3594     0.0110   32.5354   0.0000    0.3377   0.3810
treated      0.0124     0.0228    0.5441   0.5864   -0.0323   0.0571
------------------------------------------------------------------
Omnibus:             118.537       Durbin-Watson:          1.953  
Pro

# Question 3

Here are the 3 rules that should hold for IV Estimates:

- The instrument Z should be independent of U. Where U is the error term.


- The instrument Z should not affect Y when X is held constant **(exclusion restriction)**.


- The instrument Z should **not** be independent of X.

I would say that by looking at the table below, we have selected a suitable instrument.

This is because Z (assignment) was randomly assigned, in addition to Z (assignment) having no direct effect on outcomes (diabetes).

In [18]:
#correlation of outcome, instrument and endogenous regressor
mto_df[["diabetes", "assignment", "treated"]].corr()

,diabetes,assignment,treated
diabetes,1.000000,-0.011132,0.010938
assignment,-0.011132,1.000000,0.493861
treated,0.010938,0.493861,1.000000


# Question 4

The exclusion restriction is as follows:

- The instrument (assignment) should not affect Y (diabetes) when X (treated) is held constant.

A violation of this condition would be is assignment was bases on having diabetes.

The regression below shows a coefficient very close to 0 in addition to be statistically **insignificant**. This regression confirms that assignment is a suitable IV.

In [19]:
FEATURES = ['const', "assignment"]

model_2 = sm.OLS(mto_df['diabetes'], mto_df[FEATURES], missing="drop")
results_2 = model_2.fit()
print(results_2.summary2())

                 Results: Ordinary least squares
Model:              OLS              Adj. R-squared:     -0.000   
Dependent Variable: diabetes         AIC:                3402.4505
Date:               2018-03-07 14:10 BIC:                3414.0793
No. Observations:   2476             Log-Likelihood:     -1699.2  
Df Model:           1                F-statistic:        0.3066   
Df Residuals:       2474             Prob (F-statistic): 0.580    
R-squared:          0.000            Scale:              0.23119  
-------------------------------------------------------------------
                 Coef.   Std.Err.     t     P>|t|    [0.025  0.975]
-------------------------------------------------------------------
const            0.3683    0.0145  25.3684  0.0000   0.3398  0.3967
assignment      -0.0108    0.0195  -0.5537  0.5798  -0.0489  0.0274
------------------------------------------------------------------
Omnibus:             118.535       Durbin-Watson:          1.953  
Prob(Omn

# Question 5

The wald estimate is approx $-0.026$.

This has a different sign compared to the regression we ran in Question 2.

In this our estimate the the ratio of the mean difference between assignment and no-assingment to the mean difference between treated and not treated.

In [20]:
Y_Bar_Z_1 = mto_df[mto_df['assignment'] == 1]['diabetes'].mean()

Y_Bar_Z_0 = mto_df[mto_df['assignment'] == 0]['diabetes'].mean()

D_Bar_Z_1 = mto_df[mto_df['assignment'] == 1]["treated"].mean()

D_Bar_Z_0 = mto_df[mto_df['assignment'] == 0]['treated'].mean()

wald_estimate = (Y_Bar_Z_1 - Y_Bar_Z_0)/(D_Bar_Z_1 - D_Bar_Z_0)

wald_estimate

-0.025566519234902902

In [21]:
# Mean values grouped by assignment.
mto_df.groupby("assignment").mean()[['diabetes','treated']]

,diabetes,treated
assignment,,
0.0,0.368277,0.00000
1.0,0.357505,0.42132


# Question 6

In order to select the appropriate covariates, I ran individual regressions on `treated`. I have ommitted to the regression outputs for brevity. I have only included the outputs for the 2 covariates I chose.

I chose 2 co-variates that have a statistically significant relationship with `treated`.

These covariates are:

- `hispanic`

- `absence_mental_probs_z`


These are the two covariates that I will be including in 2SLS.

We can see that the instrument is strong in stage 1 as these is statistical significance!

Using Manual 2SLS, I obtained an estimate of $-0.0273$.

This is essentially the same as the manual calculation of the WALD LATE estimate in question 5.

**Note:** that while our parameter estimates are correct, our standard errors are not and for this reason, computing 2SLS "manually" (in stages with OLS) is not recommended. We will remedy this in Question 7.

In [22]:
# need to encode categorical columns with 1's and 0's
for col in mto_df.select_dtypes(include=['category']).columns:
    mto_df[col] = mto_df[col].apply(lambda x: encode(x)).astype("float64")

In [23]:
# 2 individual regressions

model = sm.OLS(mto_df['treated'], mto_df[["const","hispanic"]], missing="drop")
results = model.fit()
print(results.summary2())
print()

model = sm.OLS(mto_df['treated'], mto_df[["const","absence_mental_probs_z"]], missing="drop")
results = model.fit()
print(results.summary2())
print()

                 Results: Ordinary least squares
Model:              OLS              Adj. R-squared:     0.003    
Dependent Variable: treated          AIC:                2770.7845
Date:               2018-03-07 14:10 BIC:                2782.4133
No. Observations:   2476             Log-Likelihood:     -1383.4  
Df Model:           1                F-statistic:        8.330    
Df Residuals:       2474             Prob (F-statistic): 0.00393  
R-squared:          0.003            Scale:              0.17913  
--------------------------------------------------------------------
            Coef.    Std.Err.      t      P>|t|     [0.025    0.975]
--------------------------------------------------------------------
const       0.2534     0.0107   23.6956   0.0000    0.2324    0.2743
hispanic   -0.0509     0.0176   -2.8862   0.0039   -0.0855   -0.0163
------------------------------------------------------------------
Omnibus:             436.537       Durbin-Watson:          1.515  
Pro

In [24]:
# Stage 1: regress treated on the instrumental variable.

results_fs = sm.OLS(mto_df['treated'],
                    mto_df[['const', 'assignment']],
                    missing='drop').fit()
print(results_fs.summary())

                            OLS Regression Results                            
Dep. Variable:                treated   R-squared:                       0.244
Model:                            OLS   Adj. R-squared:                  0.244
Method:                 Least Squares   F-statistic:                     798.0
Date:                Wed, 07 Mar 2018   Prob (F-statistic):          2.06e-152
Time:                        14:10:43   Log-Likelihood:                -1041.4
No. Observations:                2476   AIC:                             2087.
Df Residuals:                    2474   BIC:                             2098.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       3.026e-15      0.011   2.72e-13      1.0

In [25]:
# store the predicted values
mto_df['predicted_treated'] = results_fs.predict()

In [26]:
# Second Stage - our estimate is essentially identical to the previous WALD estimate
results_ss = sm.OLS(mto_df['diabetes'],
                    mto_df[['const', 'predicted_treated','hispanic', 'absence_mental_probs_z']],
                   missing='drop').fit()
print(results_ss.summary2())

                   Results: Ordinary least squares
Model:               OLS               Adj. R-squared:      -0.000   
Dependent Variable:  diabetes          AIC:                 3404.6784
Date:                2018-03-07 14:10  BIC:                 3427.9360
No. Observations:    2476              Log-Likelihood:      -1698.3  
Df Model:            3                 F-statistic:         0.6921   
Df Residuals:        2472              Prob (F-statistic):  0.557    
R-squared:           0.001             Scale:               0.23121  
---------------------------------------------------------------------
                        Coef.  Std.Err.    t    P>|t|   [0.025 0.975]
---------------------------------------------------------------------
const                   0.3651   0.0165 22.0805 0.0000  0.3327 0.3975
predicted_treated      -0.0273   0.0463 -0.5893 0.5557 -0.1180 0.0635
hispanic                0.0081   0.0201  0.4034 0.6867 -0.0313 0.0475
absence_mental_probs_z  0.0122   0.0098

# Question 7

We will now recreate the same estimate as above, this time without manual calculation.

Once more, from the output below, we see identical estimate to that in question 6 and question 5.

The standard errors are a bit different as prevously mentioned.

In [27]:
iv = ivregress(dependent=mto_df['diabetes'],
            exog=mto_df[['const', "hispanic", "absence_mental_probs_z"]],
            endog=mto_df['treated'],
            instruments=mto_df['assignment']).fit(cov_type='unadjusted')

print(iv.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:               diabetes   R-squared:                     -0.0004
Estimator:                    IV-2SLS   Adj. R-squared:                -0.0016
No. Observations:                2476   F-statistic:                    2.0773
Date:                Wed, Mar 07 2018   P-value (F-stat)                0.5565
Time:                        14:10:43   Distribution:                  chi2(3)
Cov. Estimator:            unadjusted                                         
                                                                              
                                   Parameter Estimates                                    
                        Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------------------
const                      0.3654     0.0169     21.594     0.0000      0.3323      0.3986
hisp